# MOwNiT Laboratorium 2
## Zadanie 1
Napisz w dowolnym języku zwyczajną (wolną) dyskretną transformatę Fouriera.

##$$X_k = \sum_{n=0}^{N-1}x_n \cdot e^{-i2\pi kn/N}$$

In [23]:
import numpy as np

def dft(x):
    N = x.shape(0)
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = (-2j * np.pi * k * n/N)   
    return  np.dot(M, x)